In [137]:
!pip install pandas==1.5.3 matplotlib==3.7 plotly==5.13.1

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import glob
import dateutil.parser
import os
import datetime

%matplotlib inline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [138]:
!git clone --depth 1 https://github.com/Yquetzal/DefiEGC2024.git
%cd DefiEGC2024

Cloning into 'DefiEGC2024'...
remote: Enumerating objects: 925, done.
remote: Counting objects: 100% (925/925), done.
remote: Compressing objects: 100% (925/925), done.
remote: Total 925 (delta 0), reused 923 (delta 0), pack-reused 0
Receiving objects: 100% (925/925), 64.96 MiB | 10.42 MiB/s, done.
Updating files: 100% (920/920), done.
/content/DefiEGC2024/DefiEGC2024/DefiEGC2024


## About Dataset

* Time Series
  * df_global - *contains the number of transactions, payments, blocks, received and sent satoshis, fee statistics, miners reward, created coins, auto transaction*
  * df_actors - *data on 100 most active actors (per activity day) such as its identity, data on received and spent satoshis, data of fee paid, auto transactions*
  * df_external - *bitcoin price (USD) and hash rate*
* Transaction Networks
  * df_network - *network of transactions per day in the format source actor, target actor, total value and number of transactions*

## Global

We are going to read the csv file.
1. Check types of values
2. Remove unnecessary columns and rows
3. Treat the Nulls 
4. Treat the duplicates



In [139]:
df_global = pd.read_csv("timeseries/global.csv", index_col=0, parse_dates=True).sort_index()
df_global.drop(labels=["year", "month", "day"], axis=1, inplace=True)

In [140]:
df_global.head()

,total_fee,total_received_satoshi,total_sent_satoshi,mean_fee_satoshi,mean_feeUSD,mean_fee_for100,nb_transactions,nb_payments,mean_nb_inputs,mean_nb_outputs,nb_mining,total_mining_satoshi,newly_created_coins,self_spent_satoshi,total_received,volume_without_self
2015-01-01,801373971,45473028339385,45058028339385,13609.829337,4.284054e+06,0.799999,58882,161194,2.595241,2.737577,166,415801373971,415000000000,2757774885900,45473028339385,42715253453485
2015-01-02,1159934218,73370568521080,72955568521080,14978.682808,4.732406e+06,0.778933,77439,257633,2.715557,3.326915,166,416159934218,415000000000,4316724676318,73370568521080,69053843844762
2015-01-03,1196476043,82635189454578,82220189454578,14304.385766,4.086009e+06,0.797419,83644,295618,2.781885,3.534240,166,416196476043,415000000000,5584584356557,82635189454578,77050605098021
2015-01-04,2159195328,115159442449532,114726942449532,25138.199015,6.619757e+06,0.899054,85893,280288,2.770133,3.263223,173,434659195328,432500000000,6554587956125,115159442449532,108604854493407
2015-01-05,1608443040,112439228285641,112051728285641,16955.609622,4.662858e+06,0.898594,94862,286552,2.954471,3.020725,155,389108443040,387500000000,5063695492444,112439228285641,107375532793197


Next, we check dtype of the index in order to do operation on datetime and the type of other columns.

In [141]:
df_global.index.dtype

dtype('<M8[ns]')

In [142]:
df_global.dtypes

total_fee                   int64
total_received_satoshi      int64
total_sent_satoshi          int64
mean_fee_satoshi          float64
mean_feeUSD               float64
mean_fee_for100           float64
nb_transactions             int64
nb_payments                 int64
mean_nb_inputs            float64
mean_nb_outputs           float64
nb_mining                   int64
total_mining_satoshi        int64
newly_created_coins         int64
self_spent_satoshi          int64
total_received              int64
volume_without_self         int64
dtype: object

We are checking if they are some missing values in the global.csv dataframe.

In [143]:
df_global.isnull().values.any()

False

No null value were found. Now, we check if they are duplicates in the dataframe.

In [144]:
df_global.duplicated().any()

False

It seems no duplicated values were found in the dataframe. We will repeat the same action for the other dataframes.

## External

In [145]:
df_external = pd.read_csv("timeseries/external.csv", index_col="date", usecols=["date", "PriceUSD", "HashRate"], parse_dates=True).sort_index()

In [146]:
df_external.head()

,PriceUSD,HashRate
date,,
2015-01-02,315.942732,333345.017260
2015-01-03,285.647310,327284.198765
2015-01-04,263.334575,347486.927083
2015-01-05,275.003852,315162.561773
2015-01-06,287.549521,319203.107437


In [147]:
df_external.index.dtype

dtype('<M8[ns]')

In [148]:
df_external.dtypes

PriceUSD    float64
HashRate    float64
dtype: object

In [149]:
df_external.isnull().values.any()

False

In [150]:
df_external.duplicated().any()

False

## Blockchain by actors

We repeat the same step one more time for the block_chain_by_actor dataframe.

In [151]:
df_actors = pd.read_csv("timeseries/blockchain_by_actor.csv", index_col=0, parse_dates=['date']).sort_index()
df_actors.drop(labels=["year", "month", "day"], axis=1, inplace=True)
df_actors["identity"] = df_actors["identity"].astype(pd.StringDtype())

In [152]:
df_actors.head()

,identity,received,nb_received,date,sum_fee,mean_fee_for100,nb_transactions,sent,self_spent,self_spent_estimated,nb_spent,spent
0,HappyCoins.com,1.377590e+09,4.0,2015-01-01,49596.0,0.004957,4.0,1.277500e+09,0.0,0.0,8.0,1.277500e+09
1,821,2.476057e+09,5.0,2015-01-01,10000.0,0.000851,1.0,1.174552e+09,0.0,0.0,2.0,1.174552e+09
2,FortuneJack.com,1.201470e+08,29.0,2015-01-01,83000.0,0.194670,11.0,1.541127e+08,14158970.0,14158970.0,22.0,1.399538e+08
3,396,1.906135e+10,70.0,2015-01-01,540000.0,0.023738,54.0,1.842989e+10,0.0,0.0,108.0,1.842989e+10
4,SatoshiMines.com,7.401319e+08,193.0,2015-01-01,740000.0,0.447934,45.0,1.813624e+09,35044568.0,35044568.0,84.0,1.778579e+09


In [153]:
df_actors.index.dtype

dtype('int64')

In [154]:
df_actors.dtypes

identity                        string
received                       float64
nb_received                    float64
date                    datetime64[ns]
sum_fee                        float64
mean_fee_for100                float64
nb_transactions                float64
sent                           float64
self_spent                     float64
self_spent_estimated           float64
nb_spent                       float64
spent                          float64
dtype: object

In [155]:
df_actors.isnull().values.any()

False

In [156]:
df_actors.duplicated().any()

False

## Network

In [157]:
df_network = pd.DataFrame()

dfs = []

for file_path in glob.glob("networks/*"):
    date = os.path.splitext(os.path.basename(os.path.normpath(file_path)))[0]
    date = dateutil.parser.parse(date)

    df_temp = pd.read_csv(file_path)
    df_temp['date'] = date
    # set index to be date and hour
    df_temp.set_index(['date'], inplace=True)
    dfs.append(df_temp)

df_network = pd.concat(dfs, axis=0)

df_network["Source"] = df_network["Source"].astype(pd.StringDtype())
df_network["Target"] = df_network["Target"].astype(pd.StringDtype())

df_network

,Source,Target,value,nb_transactions
date,,,,
2015-11-30,50658390,ePay.info_CoinJoinMess,1257822,2
2015-11-30,Genesis-Mining.com,833,198820904,2
2015-11-30,1228047,51014,48775876,1
2015-11-30,Huobi.com,331,6723075345,10
2015-11-30,578,MercadoBitcoin.com.br,18834000,1
...,...,...,...,...
2016-09-24,0,2803,112798719,1
2016-09-24,SatoshiDice.com,Poloniex.com,4363036,2
2016-09-24,50354,52030,84386800,1


In [158]:
df_network.head()

,Source,Target,value,nb_transactions
date,,,,
2015-11-30,50658390,ePay.info_CoinJoinMess,1257822,2
2015-11-30,Genesis-Mining.com,833,198820904,2
2015-11-30,1228047,51014,48775876,1
2015-11-30,Huobi.com,331,6723075345,10
2015-11-30,578,MercadoBitcoin.com.br,18834000,1


In [159]:
df_network.index.dtype

dtype('<M8[ns]')

In [160]:
df_network.dtypes

Source             string
Target             string
value               int64
nb_transactions     int64
dtype: object

In [161]:
df_network.isnull().values.any()

False

In [162]:
df_network.reset_index().duplicated().any()

False

Actors Analysis

In [163]:
from plotly.subplots import make_subplots

df_diff = df_actors.copy()
df_diff["diff"] = df_diff["received"] - df_diff["spent"]

fig = px.line(df_diff, x='date', y='diff', line_group="identity", color='identity')
# second bitcoin halving in 2016
fig.add_vline(x=datetime.datetime(2016, 7, 9), line_width=1, line_color="red")

# Add figure title
fig.update_layout(
    title_text="Difference by Identity"
)

fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [202]:
best_range = np.abs(df_diff["diff"]).quantile(0.99).min()
print(best_range)

df_outliers = df_diff[np.abs(df_diff["diff"]) > best_range]
df_outliers["diff"] = df_outliers["diff"] / 10e8

1928240716261.786


<ipython-input-202-63d5501b70a8>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [203]:
fig = px.scatter(df_outliers, x='date', y='diff', color='identity', size="nb_transactions")
# second bitcoin halving in 2016
fig.add_vline(x=datetime.datetime(2016, 7, 9), line_width=1, line_color="red")

# Add figure title
fig.update_layout(
    title_text="Outliers by Identity"
)

fig.show()

In [208]:
df_outliers["Profit / Transaction / Day"] = df_outliers["diff"] / df_outliers["nb_transactions"]
df_outliers = df_outliers.rename(columns={"date" : "Date"})

fig = px.scatter(df_outliers, x='Date', y='Profit / Transaction / Day', color='identity')
# second bitcoin halving in 2016
fig.add_vline(x=datetime.datetime(2016, 7, 9), line_width=1, line_color="red")

# Add figure title
fig.update_layout(
    title_text="Outliers in Daily Profit in Bitcoin by Identity"
)

fig.show()

In [165]:
df_network = df_network.reset_index()

In [166]:
# select transactions with target Xapo.com from network from August 9 2015

df_transactions_xapo_in = df_network.loc[(df_network["Target"] == "Xapo.com") & (df_network["date"] == datetime.datetime(2015, 8, 9))]
df_transactions_xapo_out = df_network.loc[(df_network["Source"] == "Xapo.com") & (df_network["date"] == datetime.datetime(2015, 8, 9))]
df_transactions_xapo_out = df_transactions_xapo_out.rename(columns={'Source': 'Target', 'Target': 'Source'})
df_transaction_total = df_transactions_xapo_in.copy()
df_transaction_total = df_transactions_xapo_in.join(df_transactions_xapo_out.set_index(['Source', 'Target']), on=['Source', 'Target'], rsuffix='_out')
df_transaction_total

,date,Source,Target,value,nb_transactions,date_out,value_out,nb_transactions_out
135380,2015-08-09,Bitcoin.de,Xapo.com,1321941223,1,2015-08-09,7.102574e+07,2.0
135424,2015-08-09,73413077,Xapo.com,66483404,2,NaT,NaN,NaN
135536,2015-08-09,354,Xapo.com,28509266642,29,NaT,NaN,NaN
135560,2015-08-09,1567,Xapo.com,1292500,1,NaT,NaN,NaN
135572,2015-08-09,HolyTransaction.com,Xapo.com,11614217,8,2015-08-09,9.378714e+11,46.0
...,...,...,...,...,...,...,...,...
139773,2015-08-09,417,Xapo.com,22174418,1,NaT,NaN,NaN
139979,2015-08-09,3368,Xapo.com,4238592856,82,2015-08-09,1.149555e+08,2.0
140014,2015-08-09,PrimeDice.com,Xapo.com,10045756,4,2015-08-09,4.590141e+08,26.0
140049,2015-08-09,AnoniBet.com,Xapo.com,2250000,1,2015-08-09,1.300533e+07,1.0
